In [ ]:
!python main.py

In [ ]:
# Master reset all databases and posts
from main import reset

reset()

In [ ]:
# Only remake all .md posts
from scholar.post_maker import make_all

make_all(overwrite=True)

# Archive

Contains 1 off snippets to make change to the system.

## Add new column called arxiv_term in SQLite

In [ ]:
import sqlite3

# Reset the new columns
# with sqlite3.connect('scholar/paper.db') as conn:
#     c = conn.cursor()
#     c.execute('ALTER TABLE paper DROP COLUMN arxiv_term_short;')
#     c.execute('ALTER TABLE paper DROP COLUMN arxiv_term_long;')
#     conn.commit()

with sqlite3.connect("scholar/paper.db") as conn:
    c = conn.cursor()
    c.execute("ALTER TABLE paper ADD COLUMN category VARCHAR(255);")
    conn.commit()

## Compute new column

In [ ]:
# import sqlite3
# import pandas as pd
# from scholar.database import Paper

# with sqlite3.connect("scholar/paper.db") as conn:

#     # Add journal column
#     cursor = conn.cursor()
#     cursor.execute("ALTER TABLE paper ADD COLUMN journal VARCHAR(255);")

#     # Export data to pandas for easier manipulation
#     df = pd.read_sql_query("SELECT * FROM paper", conn)


# # Extract journal name from publication_info_summary
# journals = df.publication_info_summary.str.split(" - ", expand=True)[2]

# # Update the database
# for record, new_value in zip(Paper.select(), journals):
#     record.journal = new_value
#     record.save()

## Extract from biorxiv API

In [8]:
from scholar.database import Paper
from scholar.api import to_doi, query_biorxiv

In [4]:
papers = Paper.select().where(Paper.journal == "biorxiv.org")

In [9]:
for paper in papers:
    print(paper.link)
    paper.doi = to_doi(paper.link)

    data = query_biorxiv(paper.doi)
    paper.category = data["category"]
    paper.save()
    



https://www.biorxiv.org/content/10.1101/2023.04.17.537118.abstract
https://www.biorxiv.org/content/10.1101/2023.03.06.531327.abstract
https://www.biorxiv.org/content/10.1101/2023.03.02.530774.abstract
https://www.biorxiv.org/content/10.1101/2023.03.01.530704.abstract
https://www.biorxiv.org/content/10.1101/2023.02.27.530253.abstract
https://www.biorxiv.org/content/10.1101/2023.01.31.526269.abstract
https://www.biorxiv.org/content/10.1101/2023.01.11.523690.abstract
https://www.biorxiv.org/content/10.1101/2023.01.09.523219.abstract
https://www.biorxiv.org/content/10.1101/2023.01.09.523230.abstract
